In [1435]:
import pandas as pd
import json
import networkx as nx
from networkx_query import search_nodes, search_edges
import numpy as np

In [1436]:
with open('/Users/theojolliffe/Documents/Wayback BBC/topicScoresOrder.json', 'r') as fp:
    topicScoresOrder = json.load(fp)
topicScoresOrder = {i: {i[0]: 1/(1+i[1]/10) for i in topicScoresOrder[i]} for i in topicScoresOrder}

In [1437]:
bbcReg = pd.read_csv('/Users/theojolliffe/Downloads/bbc_regions.csv')
bbcRegLU = {}
for i in bbcReg.index:
    if bbcReg['areaCode'].iloc[i] in [lad['code'] for lad in lads]:
        bbcRegLU[bbcReg['areaCode'].iloc[i]] = bbcReg['bbc_online'].iloc[i]
bbcRegLU['E06000053'] = 'cornwall'
bbcRegLU['E06000060'] = 'beds_bucks_and_herts'
bbcRegLU['E09000001'] = 'london'
bbcRegLU['E07000150'] = 'northamptonshire'

regions = ['cumbria', 'lancashire', 'merseyside', 'manchester', 'tees', 'tyne_and_wear', 'Humberside', 'york_and_north_yorkshire', 'leeds_and_west_yorkshire', 'lincolnshire', 'south_yorkshire', 'birmingham_and_black_country', 'coventry_and_warwickshire', 'hereford_and_worcester', 'shropshire', 'stoke_and_staffordshire', 'derbyshire', 'leicester', 'northamptonshire', 'nottingham', 'bristol', 'cornwall', 'devon', 'gloucestershire', 'somerset', 'wiltshire', 'beds_bucks_and_herts', 'cambridgeshire', 'essex', 'norfolk', 'suffolk', 'berkshire', 'dorset', 'hampshire', 'oxford', 'kent', 'london', 'surrey', 'sussex']
bbcNames = {
     'derby': 'derbyshire', 
     'hereford and worcester': 'hereford_and_worcester',
     'tyne and wear': 'tyne_and_wear',
     'northampton': 'northamptonshire',
     'liverpool': 'merseyside',
     'coventry and warwickshire': 'coventry_and_warwickshire',
     'humberside': 'Humberside',
     'sheffield and south yorkshire': 'south_yorkshire',
     'hampshire & isle of wight': 'hampshire',
     'stoke and staffordshire': 'stoke_and_staffordshire',
     'york & north yorkshire': 'york_and_north_yorkshire',
     'birmingham and black country': 'birmingham_and_black_country',
     'beds, bucks and herts': 'beds_bucks_and_herts',
     'leeds and west yorkshire': 'leeds_and_west_yorkshire'
}
for i in bbcRegLU.keys():
    bbcRegLU[i] = bbcRegLU[i].lower()
    if bbcRegLU[i] not in regions:
        try:
            bbcRegLU[i] = bbcNames[bbcRegLU[i]]
        except:
            pass

In [1438]:
tfidf = pd.read_csv('/Users/theojolliffe/Documents/Wayback BBC/BBCRegionalTFIDF.csv')
tfidf = tfidf.set_index('Unnamed: 0')
for i in tfidf.columns:
    for j in tfidf[i].index:
        tfidf[i].loc[j] = 1+100*tfidf[i].loc[j]
tfidfLU = {}
for i in regions:
    tfidfLU[i]=tfidf.loc[i].to_dict()


In [1439]:
TFtopicLU = {
    'health': 'health',
    'racial': 'ethnicity',
    'population': 'population',
    'age': 'agemed',
    'work': 'economic',
    'commuter': 'travel',
    'housing': 'tenure'
}
TFtopicRev = dict((TFtopicLU[x],x) for x in TFtopicLU)

In [1440]:
options = pd.read_csv('./csv/lists/places_2020.csv')
areaType = pd.read_csv('/Users/theojolliffe/Documents/Census Data/censusAreaLookup.csv')
typeLookup = {}
for i in areaType.index:
    typeLookup[areaType.iloc[i]['Name']]=areaType.iloc[i]['Group name']

In [1441]:
areasO = []
for i in options["code"]:
    try:
        areasO.append(json.load(open(f'/Users/theojolliffe/Documents/Census Data/census-data-main/json/place/{i}.json', 'rb')))
    except FileNotFoundError:
        print(i)
        pass
    
for area in areasO:
    area['data']['agemed']['value']['change']['all'] = area['data']['agemed']['value']['2011']['all']-area['data']['agemed']['value']['2001']['all']

E05009289
E05009290
E05009293
E05009294
E05009295
E05009297
E05009298
E05009299
E05009300
E05009301
E05009303
E05009306
E05009307
E05009312
E05011090


In [1442]:
areas = []
for i in options["code"]:
    try:
        areas.append(json.load(open(f'/Users/theojolliffe/Documents/pipeline_2/final/json/place/{i}.json', 'rb')))
    except FileNotFoundError:
#         print(i)
        pass
    
for area in areas:
    area['data']['agemed']['value']['change']['all'] = area['data']['agemed']['value']['2011']['all']-area['data']['agemed']['value']['2001']['all']

In [1443]:
# Seperate the areas by area type
regions = []
for i in areas:
    if i['type']=='rgn':
        regions.append(i)
        
lads = []
for i in areas:
    if i['type']=='lad':
        lads.append(i)
        
countries = []
for i in areas:
    if (i['type']=='ew')|(i['type']=='ctry'):
        countries.append(i)

In [1444]:
# Find each LAD local and national ranks and put in prioirity order
for thisLad in lads:
    if thisLad['parents'][0]['type'] == "cty":
        thisLad['parents'] = thisLad['parents'][1:]
    
    # Empty array will be populted with ranks for each variable
    ranks = []

    # Filter areas with same parent
    sister_lads = []
    lad_code = thisLad['parents'][0]['code']
    for i in range(len(lads)):
        if  lad_code == lads[i]['parents'][0]['code']:
            sister_lads.append(lads[i])

    # Loop through the various data variables
    for a in thisLad['data']:
        if a in ['population', 'density', 'agemed']:
            b = 'value'
        else:
            b = 'perc'
        # Create nested object with localised rank
        thisLad['data'][a][b+"_rank_local"] = {}    
        for c in ['2011', 'change']:
            thisLad['data'][a][b+"_rank_local"][c] = {}
            for d in thisLad['data'][a][b][c]:
                vari = thisLad['data'][a][b][c][d]

                # Create sorted list of values from sister areas
                group_values = []
                for lad in sister_lads:
                    group_values.append(lad['data'][a][b][c][d])
                    group_values = [x if (type(x) == float) | (type(x) == int) else np.nan for x in group_values]
                group_values.sort(reverse=True)

                # Find index of value of area of interest
                varRank = group_values.index(vari) + 1

                # Convert bottom half rankings into negative values
                if varRank>len(group_values)/2:
                    varRank = varRank-len(group_values)-1
                    
                natRank = thisLad['data'][a][b+"_rank"][c][d]
                if natRank > 168:
                    natRank = natRank-336-1

                thisLad['data'][a][b+"_rank_local"][c][d] = varRank
                thisLad['data'][a][b+"_rank"][c][d] = natRank

                # Append ranking data to original array
                ranks.append({
                    'label': a+'_'+b+'_'+c+'_'+d, 
                    'locRank': varRank, 
                    'natRank': natRank, 
                    'value': vari})

    # Sort in rank order
    ranks = sorted(ranks, key=lambda x: (abs(x['locRank']), -abs(x['value'])))
    thisLad["Priorities"] = ranks

In [1445]:
topics = [{1: 'population', 2: 'value', 3: 'all'}, 
          {1: 'agemed', 2: 'value', 3: 'all'},
          {1: 'health', 2: 'perc', 3: 'good'},
          {1: 'travel', 2: 'perc', 3: 'car_van'},
          {1: 'tenure', 2: 'perc', 3: 'rented_social'},
          {1: 'tenure', 2: 'perc', 3: 'rented_private'},
          {1: 'tenure', 2: 'perc', 3: 'owned'},
          {1: 'agemed', 2: 'value', 3: 'all'},
          {1: 'economic', 2: 'perc', 3: 'employee'},
#           {1: 'religion', 2: 'perc', 3: 'Christian'},
          {1: 'health', 2: 'perc', 3: 'bad'},
          {1: 'ethnicity', 2: 'perc', 3: 'black'},
          {1: 'ethnicity', 2: 'perc', 3: 'white'},
          {1: 'ethnicity', 2: 'perc', 3: 'asian'},
          {1: 'economic', 2: 'perc', 3: 'inactive'},
          {1: 'economic', 2: 'perc', 3: 'self-employed'}]

ladCodes = []
for lad in lads:
    ladCodes.append(lad['code'])
    
    
df = pd.DataFrame(columns=
                  ['lad', '2001',  '2011',  'change', 'natRank', 'localRank', 'topic', 'parent'])
for j in range(len(topics)):
    df1 = pd.DataFrame(index=ladCodes, columns=
                      ['lad', '2001',  '2011',  'change', 'natRank', 'localRank', 'topic', 'parent'])
    for i in range(len(lads)):
        df1['lad'].iloc[i] = lads[i]['name']
        df1['2001'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2001'][topics[j][3]]
        df1['2011'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2011'][topics[j][3]]    
        df1['change'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['change'][topics[j][3]]
        df1['natRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank"]['change'][topics[j][3]]    
        df1['localRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank_local"]['change'][topics[j][3]]    
        df1['topic'].iloc[i] = topics[j][1]+"_"+topics[j][3]
        df1['parent'].iloc[i] = lads[i]['parents'][0]['name']
    df = pd.concat([df,df1])
df['natRank'] = np.where(df['natRank'] > 168, df['natRank']-337,df['natRank'])



for topic in topics:
    for lad in lads:
        
#       Find the areas that this area has overtaken
        v2001 = lad['data'][topic[1]][topic[2]]['2001'][topic[3]]
        v2011 = lad['data'][topic[1]][topic[2]]['2011'][topic[3]]
        dfT = df[(df['topic']==topic[1]+"_"+topic[3])&(df['2001']>v2001)&(df['2011']<v2011)]
        obje = []
        for i in range(dfT.shape[0]):
            obje.append(dfT.iloc[i]['lad'])
        
        if 'overtake' not in lad['data'][topic[1]][topic[2]+'_rank'].keys():
            lad['data'][topic[1]][topic[2]+'_rank']['overtake'] = {}
        lad['data'][topic[1]][topic[2]+"_rank"]['overtake'][topic[3]] = obje
        
        
        # Find areas within the region that this area has overtaken
        reg = lad['parents'][0]['name']
        dfTr = df[(df['topic']==topic[1]+"_"+topic[3])&(df['2001']>v2001)&(df['2011']<v2011)&(df['parent']==reg)]
        objeR = []
        for i in range(dfTr.shape[0]):
            objeR.append(dfTr.iloc[i]['lad'])

        
        
#       Find the area immediatly above or below
        below = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]
        above = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]-2
        if below<0:
            below=below+1
            above=above+1
        name_above = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[above]['lad']
        name_below = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[below]['lad']

        area_above = {'name': name_above,
                         'value': df[(df['lad']==name_above)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}
        area_below = {'name': name_below,
                          'value': df[(df['lad']==name_below)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}

        
        if 'above_below' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'] = {}

        if topic[3] not in lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]] = {}
        
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["above"] = area_above
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["below"] = area_below
            
        # Add top and bottom three biggest movers for each subject to every area
        df_topic = df[(df['topic']==topic[1]+"_"+topic[3])&(abs(df['natRank'])<4)]
        ob = {}
        for index, row in df_topic.iterrows():
            ob[row['natRank']]= {row['lad']: row['change']}
        if 'top_bottom' not in lad['data'][topic[1]][topic[2]+'_rank'].keys():
            lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'] = {}
        lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'][topic[3]]=ob

In [1446]:
# Filter out priority list by subject
for lad in lads:
    subjectList = ["fair", "rent_free", "shared_ownership", "bicycle", "taxi", "moto", "bus", "other", "female", "male", "student"]
    priorities = []
    priorities2011 = []
    for rank in lad['Priorities']:
        s=rank['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        if ((s[2] == "change") & (s[3] not in subjectList)):
            priorities.append(rank)
        if (s[2]!="change"):
            priorities2011.append(rank)
    lad['pri'] = priorities
    lad['pri2011'] = priorities2011

In [1447]:
# Create triple data for areas of closest proximity
geogTriples = []
for lad in lads:
    list1 = lad['bounds'][0]+lad['bounds'][1]
    for lad2 in lads:
        list2 = lad2['bounds'][0]+lad2['bounds'][1]
        listDif = [abs(list1[i]-list2[i]) for i in [0,1,2,3]]
        listDif.sort()
        if (sum(listDif[:3]) < 0.3) & (lad!=lad2):
            geogTriples.append([lad['name'], lad2['name'], ("near", round(sum(listDif[:3]), 2))])


In [1456]:
for thisLad in lads:

    # Find nearby area of same area type
    nearSimilar = []
    for i in geogTriples:
        try:
            if ((i[0]==thisLad['name'])&(typeLookup[i[1]]==typeLookup[thisLad['name']])):
                nearSimilar.append([i[0], i[1], ('near_similar', i[2][1])])
        except:
            pass

    if len(nearSimilar)==0:
        nearSimilar = [i for i in geogTriples if i[0]==thisLad['name']]

    nearSimilar = sorted(nearSimilar, key=lambda x: x[2][1])


    # Find and refine stories
    stories = []
    stories=stories+[i for i in thisLad['pri'] if abs(i['locRank']) <= 5]
    stories=stories+[i for i in thisLad['pri'] if (abs(i['natRank']) <= 10)&(abs(i['locRank']) > 5)]

    change = sorted(thisLad['pri'], reverse=True, key=lambda x: abs(x['value']))

    bigchange = [i for i in change if abs(i['value'])>4.5]
    for i in range(len(bigchange)):
        if len(stories)<4:
            stories=stories+[bigchange[i]]

    smallchange = sorted([i for i in change if abs(i['value'])<1], reverse=False, key=lambda x: abs(x['value']))
    for i in range(len(smallchange)):
        if len(stories)<5:
            stories=stories+[smallchange[i]]

    stories = sorted(stories, key=lambda x: -abs(x['value']))

    storiesRefined =[]
    notInc = ['density', 'age10yr']
    for i in stories:
        if i['label'].split("_")[0] not in notInc:
            storiesRefined.append(i)
            notInc.append(i['label'].split("_")[0])

    if (thisLad['code'][0]!='W'):
        storiesRefined = sorted(storiesRefined, key=lambda x: (abs(x['locRank'])*(topicScoresOrder[bbcRegLU[thisLad['code']]][x['label'].split('_')[0]])*(abs(x['natRank'])/10)*(5/(1+abs(x['value'])))))
        storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['label']=='population_value_change_all')

    
    #Find differences in data
    if (thisLad['parents'][0]['name']=='Wales'):                        
        region=[i for i in countries if i['name']=="Wales"][0]
        country=[i for i in countries if i['name']=="Wales"][0]
    else:
        region=[i for i in regions if i['name']==thisLad['parents'][0]['name']][0]
        country=[i for i in countries if i['name']==thisLad['parents'][1]['name']][0]

    def reg(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return region['data'][s[0]][s[1]][s[2]][s[3]]
    regDiff = [(i['label'], i['value']-reg(i)) for i in change]
    regDiff = sorted(regDiff, reverse=True, key=lambda x: abs(x[1]))

    def cou(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return countries[1]['data'][s[0]][s[1]][s[2]][s[3]]
    couDiff = [(i['label'], i['value']-cou(i)) for i in change]
    couDiff = sorted(couDiff, reverse=True, key=lambda x: abs(x[1]))
    
    try:
        nearSimilarData = {}
        nearSimilarData['name'] = nearSimilar[0][1]
        nearSimilarData['data'] = [lad for lad in lads if lad['name']==nearSimilar[0][1]][0]['data']
    except:
        nearSimilarData = []

    def near(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return nearSimilarData['data'][s[0]][s[1]][s[2]][s[3]]
    if len(nearSimilarData)>0:
        nearDiff = [(i['label'], i['value']-near(i)) for i in change]
        nearDiff = sorted(nearDiff, reverse=True, key=lambda x: abs(x[1]))

    nearTops = sorted([(lad['name'], lad['pri'][0]) for lad in lads if lad['name'] in [i[1] for i in nearSimilar if i[0]==thisLad['name']]], key=lambda x: abs(x[1]['locRank']))

    ageBandChange = sorted([(i, thisLad['data']['age10yr']['value']['2011'][i]-thisLad['data']['age10yr']['value']['2001'][i]) for i in thisLad['data']['age10yr']['value']['2001'].keys()], 
           reverse=True, key=lambda x: x[1])

    equalAgeChange = [lad['name'] for 
     lad in lads if (lad['parents'][0]['name'] == thisLad['parents'][0]['name']) 
     & (lad['data']['agemed']['value']['change']['all']==thisLad['data']['agemed']['value']['change']['all'])]
    len(equalAgeChange)
    
    # Create a varibale 'hiRank' which holds the index of a top 5 rank at the national or reginal level to be used for an overtake story
    hiRank ={}
    hiRankNat = []
    for i in storiesRefined:
        if i['label'] not in ['population_value_change_all', 'agemed_value_change_all']:
            s=i['label'].split("_")
            if len(s)>4:
                s[3] = s[3] + "_" + s[4]
            hiRankNat.append(abs(thisLad['data'][s[0]][s[1]+"_rank"]['2011'][s[3]]))
        else:
            hiRankNat.append(100)
    print(min(hiRankNat))

    hiRankReg = []
    for i in storiesRefined:
        if i['label'] not in ['population_value_change_all', 'agemed_value_change_all']:
            s=i['label'].split("_")
            if len(s)>4:
                s[3] = s[3] + "_" + s[4]
            hiRankReg.append(abs(thisLad['data'][s[0]][s[1]+"_rank_local"]['2011'][s[3]]))
        else:
            hiRankReg.append(100)
    print(min(hiRankLoc))
    if min(hiRankNat)<5:
        hiRank['rankIn'] = hiRankLoc.index(min(hiRankLoc))
        hiRank['type'] = 'nat'
    elif min(hiRankReg)<5:
        hiRank['rankIn'] = hiRankReg.index(min(hiRankReg))
        hiRank['type'] = 'reg'
    thisLad['hiRank']=hiRank

    # Add data to object
    thisLad['hiRank']=hiRank
    thisLad['stories'] = storiesRefined
    thisLad['nearSimilar'] = {}
    thisLad['nearSimilar']['triples'] = nearSimilar
    thisLad['nearSimilar']['nearTops'] = nearTops
    thisLad['nearSimilar']['nearTops'] = nearSimilarData
    thisLad['differences'] = {}
    thisLad['differences']['country'] = couDiff
    thisLad['differences']['region'] = regDiff
    thisLad['differences']['near'] = nearDiff
    thisLad['data']['age10yr']['absChange'] = ageBandChange
    thisLad['data']['agemed']['value_rank_local']['equalAgeChange]'] = len(equalAgeChange)

25.0
1
4.0
1
10.0
1
39.0
1
73.0
1
11.0
1
100
1
3.0
1
22.0
1
43.0
1
41.0
1
17.0
1
80.0
1
11.0
1
47.0
1
7.0
1
55.0
1
3.0
1
26.0
1
94.0
1
14.0
1
69.0
1
9.0
1
30.0
1
16.0
1
35.0
1
25.0
1
5.0
1
63.0
1
19.0
1
46.0
1
42.0
1
72.0
1
2.0
1
20.0
1
34.0
1
11.0
1
11.0
1
4.0
1
12.0
1
12.0
1
22.0
1
22.0
1
22.0
1
20.0
1
97.0
1
80.0
1
46.0
1
30.0
1
1.0
1
61.0
1
58.0
1
42.0
1
16.0
1
100
1
49.0
1
15.0
1
13.0
1
23.0
1
48.0
1
14.0
1
17.0
1
2.0
1
15.0
1
28.0
1
2.0
1
1.0
1
20.0
1
34.0
1
12.0
1
27.0
1
9.0
1
21.0
1
56.0
1
6.0
1
3.0
1
18.0
1
45.0
1
8.0
1
10.0
1
2.0
1
24.0
1
4.0
1
3.0
1
35.0
1
15.0
1
60.0
1
14.0
1
17.0
1
30.0
1
59.0
1
31.0
1
2.0
1
45.0
1
70.0
1
22.0
1
12.0
1
15.0
1
7.0
1
32.0
1
17.0
1
12.0
1
9.0
1
13.0
1
33.0
1
9.0
1
68.0
1
53.0
1
29.0
1
44.0
1
34.0
1
100
1
1.0
1
34.0
1
15.0
1
1.0
1
41.0
1
11.0
1
47.0
1
68.0
1
33.0
1
58.0
1
19.0
1
42.0
1
40.0
1
3.0
1
27.0
1
94.0
1
60.0
1
149.0
1
37.0
1
26.0
1
64.0
1
28.0
1
58.0
1
51.0
1
53.0
1
26.0
1
17.0
1
47.0
1
85.0
1
33.0
1
26.0
1
4.0
1
20.0
1
50.0
1
7.0
1
8

In [ ]:
[i for i in regions if i['name']==thisLad['parents'][0]['name']]

In [ ]:
manchester['parents']

In [ ]:
regions[0]['name']

In [1461]:
manchester['data']['population']['value_rank']['top_bottom']['all']

{-1: {'Barrow-in-Furness': -4.02},
 2: {'Manchester': 28.08},
 -2: {'Sefton': -3.24},
 -3: {'South Tyneside': -3.05},
 3: {'Newham': 26.28},
 1: {'Tower Hamlets': 29.57}}

In [1457]:
for lad in lads:
    try:
        with open('/Users/theojolliffe/Documents/census-data-transformed/json/place/'+lad['code']+'.json', 'w') as outfile:
            json.dump(lad, outfile)
    except:
        print("Failed: ", lad['name'])


In [1458]:
!git add .
!git commit -m "Add files"
!git push -u origin main

[main 7163490] Add files
 330 files changed, 401 insertions(+), 752 deletions(-)
Enumerating objects: 572, done.
Counting objects: 100% (572/572), done.
Delta compression using up to 16 threads
Compressing objects: 100% (334/334), done.
Writing objects: 100% (334/334), 1.19 MiB | 1.66 MiB/s, done.
Total 334 (delta 311), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (311/311), completed with 237 local objects.
To https://github.com/theojolliffe/census-data.git
   115e786..7163490  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
